## Task-1

In [ ]:
# =========================================
# Sentiment140 Dataset Loading + Preprocessing (Colab, fixed 'punkt_tab')
# =========================================
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time

# --- Authenticate with Kaggle ---
print("--- Authenticating with Kaggle ---")
os.makedirs("/root/.kaggle", exist_ok=True)

if os.path.exists("/content/kaggle.json"):
    os.system("cp /content/kaggle.json /root/.kaggle/")
else:
    raise FileNotFoundError("❌ kaggle.json not found in /content/. Please upload it first!")

os.chmod("/root/.kaggle/kaggle.json", 0o600)
print("✅ Kaggle API key configured successfully.")

# --- Download Sentiment140 dataset ---
print("\n--- Downloading Sentiment140 dataset ---")
os.system("kaggle datasets download -d kazanova/sentiment140 -p /content --force")
os.system("unzip -o /content/sentiment140.zip -d /content/")

# --- Initialize tqdm ---
tqdm.pandas()

# --- NLTK Resource Download (with punkt_tab fix) ---
for pkg in ['stopwords', 'punkt', 'punkt_tab', 'wordnet']:
    try:
        nltk.data.find(f'corpora/{pkg}')
    except LookupError:
        nltk.download(pkg, quiet=True)

# --- Step 1: Load and Explore Data ---
print("\n--- 1. Data Loading & Exploration ---")
COL_NAMES = ['target', 'ids', 'date', 'flag', 'user', 'text']
FILEPATH = '/content/training.1600000.processed.noemoticon.csv'

try:
    df = pd.read_csv(FILEPATH, encoding='ISO-8859-1', names=COL_NAMES)
    print("✅ Data loaded successfully.")
    df = df.drop_duplicates(subset=['text', 'target'], keep='first')
    print(f"After removing duplicates: {df.shape}")
    print("Target distribution:")
    print(df['target'].value_counts(normalize=True))
except FileNotFoundError:
    raise FileNotFoundError("❌ Sentiment140 file not found after download!")

# --- Define Preprocessing Functions ---
def basic_text_cleaning(text):
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@[^\s]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_lemmatize_stopwords(text, stop_words, lemmatizer):
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stop_words and len(tok) > 2]
    return ' '.join(lemmas)

# --- Preprocessing Pipeline ---
print("\n--- 2–4. Preprocessing Pipeline ---")
start_time = time.time()
df['text_cleaned'] = df['text'].progress_apply(basic_text_cleaning)
print(f"✅ Basic cleaning done in {time.time() - start_time:.2f}s")

stop_words = set(stopwords.words('english'))
stop_words.update(['im', 'u', 'ur', 'like', 'get', 'go', 'dont'])
lemmatizer = WordNetLemmatizer()

start_time = time.time()
df['text_final'] = df['text_cleaned'].progress_apply(
    lambda t: tokenize_lemmatize_stopwords(t, stop_words, lemmatizer)
)
print(f"✅ Tokenization + Lemmatization done in {time.time() - start_time:.2f}s")

df['target_encoded'] = df['target'].replace(4, 1)
df = df[df['text_final'] != '']
print(f"✅ Final dataset shape: {df.shape}")

# --- Step 5: Split Data ---
print("\n--- 5. Data Splitting (70/15/15) ---")
X = df['text_final']
y = df['target_encoded']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
print("\n✅ Dataset ready for TF-IDF or model training.")
print("\nExample Tweet:")
print("Original:", df.iloc[0]['text'])
print("Cleaned:", df.iloc[0]['text_cleaned'])
print("Final:", df.iloc[0]['text_final'])


--- Authenticating with Kaggle ---
✅ Kaggle API key configured successfully.

--- Downloading Sentiment140 dataset ---

--- 1. Data Loading & Exploration ---
✅ Data loaded successfully.
After removing duplicates: (1583691, 6)
Target distribution:
target
4    0.501048
0    0.498952
Name: proportion, dtype: float64

--- 2–4. Preprocessing Pipeline ---


100%|██████████| 1583691/1583691 [00:17<00:00, 90692.15it/s]


✅ Basic cleaning done in 17.51s


100%|██████████| 1583691/1583691 [02:34<00:00, 10272.52it/s]


✅ Tokenization + Lemmatization done in 154.24s
✅ Final dataset shape: (1574120, 9)

--- 5. Data Splitting (70/15/15) ---
Train: 1101884 | Val: 236118 | Test: 236118

✅ Dataset ready for TF-IDF or model training.

Example Tweet:
Original: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
Cleaned: awww thats a bummer you shoulda got david carr of third day to do it d
Final: awww thats bummer shoulda got david carr third day


In [ ]:
df.to_csv('processed_sentiment140.csv', index=False)
print("DataFrame saved to processed_sentiment140.csv")

DataFrame saved to processed_sentiment140.csv


## Task-2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


df=pd.read_csv('/content/processed_sentiment140.csv')
sample_df = df.sample(n=10000, random_state=42)
texts = sample_df["text_final"].astype(str)
labels = sample_df["target_encoded"]


vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = vectorizer.fit_transform(texts)


print("✅ TF-IDF matrix shape:", X_tfidf.shape)


try:
  pca = PCA(n_components=2, random_state=42)
  X_pca = pca.fit_transform(X_tfidf.toarray()[:500])
  plt.figure(figsize=(7,5))
  plt.scatter(X_pca[:,0], X_pca[:,1], c=labels[:500], cmap='coolwarm', alpha=0.5)
  plt.title("PCA Visualization of TF-IDF Vectors (500 Samples)")
  plt.xlabel("Principal Component 1")
  plt.ylabel("Principal Component 2")
  plt.show()
except Exception as e:
  print(f"⚠️ PCA visualization skipped: {e}")


print("Top 10 TF-IDF features:")
print(vectorizer.get_feature_names_out()[:10])


sample_df.to_csv('sample_df.csv', index=False)
print("DataFrame saved to sample_df.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/processed_sentiment140.csv'

## Task-3

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score, classification_report


sample_df = pd.read_csv('/content/sample_df.csv')
texts = sample_df["text_final"].astype(str)
labels = sample_df["target_encoded"].astype(int)


max_words = 5000
max_len = 50


tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')


X_train, X_temp, y_train, y_temp = train_test_split(padded, labels, test_size=0.3, random_state=42, stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


model = Sequential([
Embedding(max_words, 64, input_length=max_len),
Bidirectional(LSTM(64, return_sequences=False)),
Dropout(0.4),
Dense(32, activation='relu'),
Dropout(0.3),
Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


history = model.fit(
X_train, y_train,
validation_data=(X_val, y_val),
epochs=3,
batch_size=64,
verbose=1
)


y_pred = (model.predict(X_test) > 0.5).astype(int)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print(f"Accuracy: {acc:.4f}")
print(f"F1-Score: {f1:.4f}")
print(classification_report(y_test, y_pred))


import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy over Epochs')
plt.legend()


plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss over Epochs')
plt.legend()
plt.show()


model.summary()

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_df.csv'

## Task-4

In [ ]:
# =========================================
# ✅ Task 4 — Feature Selection using Ant Colony Optimization (ACO)
# =========================================
import numpy as np
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

print("\n--- Task 4: Feature Selection using Ant Colony Optimization (ACO) ---")

# ----------------------------------------------------
# Step 1️⃣ Load data (same as used in Task 3)
# ----------------------------------------------------
sample_df = pd.read_csv('/content/sample_df.csv')

texts = sample_df["text_final"].astype(str)
labels = sample_df["target_encoded"].astype(int)

# Train/Val/Test Split (same ratios)
X_train_texts, X_temp_texts, y_train, y_temp = train_test_split(texts, labels, test_size=0.3, random_state=42, stratify=labels)
X_val_texts, X_test_texts, y_val, y_test = train_test_split(X_temp_texts, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# ----------------------------------------------------
# Step 2️⃣ TF-IDF Representation
# ----------------------------------------------------
max_features = 30000  # keep it reasonable for ACO
tfidf = TfidfVectorizer(max_features=max_features, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train_texts)
X_val_tfidf = tfidf.transform(X_val_texts)
X_test_tfidf = tfidf.transform(X_test_texts)
feature_names = np.array(tfidf.get_feature_names_out())

# ----------------------------------------------------
# Step 3️⃣ Define helper: evaluate subset accuracy
# ----------------------------------------------------
def evaluate_subset(subset_indices, X_train, y_train, X_val, y_val):
    """Train a logistic regression model on a subset of features and return validation accuracy."""
    if len(subset_indices) == 0:
        return 0
    model = LogisticRegression(max_iter=300)
    model.fit(X_train[:, subset_indices], y_train)
    preds = model.predict(X_val[:, subset_indices])
    return accuracy_score(y_val, preds)

# ----------------------------------------------------
# Step 4️⃣ Define ACO function (no classes)
# ----------------------------------------------------
def ant_colony_feature_selection(n_features, X_train, y_train, X_val, y_val, n_ants=5, n_iter=3, evap_rate=0.3):
    pheromone = np.ones(n_features)
    best_subset, best_acc = [], 0

    for it in range(n_iter):
        print(f"\nIteration {it+1}/{n_iter}")
        all_subsets, all_accuracies = [], []

        for ant in range(n_ants):
            probs = pheromone / np.sum(pheromone)
            chosen = np.random.rand(n_features) < probs
            subset = np.where(chosen)[0]

            # Always ensure a minimum subset size
            if len(subset) < 5:
                subset = np.random.choice(n_features, size=10, replace=False)

            acc = evaluate_subset(subset, X_train, y_train, X_val, y_val)
            all_subsets.append(subset)
            all_accuracies.append(acc)
            print(f"🐜 Ant {ant+1}: selected {len(subset)} features, acc={acc:.4f}")

        # Best ant in this iteration
        best_idx = np.argmax(all_accuracies)
        iteration_best_subset = all_subsets[best_idx]
        iteration_best_acc = all_accuracies[best_idx]

        # Pheromone update
        pheromone = (1 - evap_rate) * pheromone
        pheromone[iteration_best_subset] += evap_rate * iteration_best_acc

        # Update global best
        if iteration_best_acc > best_acc:
            best_acc = iteration_best_acc
            best_subset = iteration_best_subset

        print(f"Iteration best acc: {iteration_best_acc:.4f}, Global best acc: {best_acc:.4f}")

    return best_subset, best_acc

# ----------------------------------------------------
# Step 5️⃣ Run ACO
# ----------------------------------------------------
start_time = time.time()
selected_indices, aco_best_acc = ant_colony_feature_selection(
    X_train_tfidf.shape[1],
    X_train_tfidf, y_train,
    X_val_tfidf, y_val,
    n_ants=5, n_iter=3
)
end_time = time.time()

selected_features = feature_names[selected_indices]
print(f"\n✅ ACO selected {len(selected_features)} features")
print("Example selected features:", selected_features[:20])
print(f"ACO Time: {end_time - start_time:.2f}s")
print(f"ACO Validation Accuracy: {aco_best_acc:.4f}")

# ----------------------------------------------------
# Step 6️⃣ Compare Model Performance Before vs After ACO
# ----------------------------------------------------
# Before ACO (all features)
base_model = LogisticRegression(max_iter=300)
base_model.fit(X_train_tfidf, y_train)
base_preds = base_model.predict(X_test_tfidf)
base_acc = accuracy_score(y_test, base_preds)
base_f1 = f1_score(y_test, base_preds)

# After ACO (selected features)
aco_model = LogisticRegression(max_iter=300)
aco_model.fit(X_train_tfidf[:, selected_indices], y_train)
aco_preds = aco_model.predict(X_test_tfidf[:, selected_indices])
aco_acc = accuracy_score(y_test, aco_preds)
aco_f1 = f1_score(y_test, aco_preds)

print("\n--- Final Model Comparison ---")
print(f"Before ACO: Accuracy={base_acc:.4f}, F1={base_f1:.4f}")
print(f"After  ACO: Accuracy={aco_acc:.4f}, F1={aco_f1:.4f}")
print(f"ACO Selected {len(selected_features)} / {len(feature_names)} features ({len(selected_features)/len(feature_names)*100:.1f}%)")
